## Test FF methods

In [ ]:
# script to define functions_folder for loading and standardizing fish movies
import sys

sys.path.append("/net/trapnell/vol1/home/nlammers/projects/data/morphseq/src")

import os
import numpy as np
from PIL import Image
from skimage import (exposure, feature, filters, io, measure,
                      morphology, restoration, segmentation, transform,
                      util)
import matplotlib
from tqdm import tqdm
from PIL import Image
import glob2 as glob
# from src.functions.image_utils import doLap
# from src.functions.utilities import path_leaf
from skimage.transform import resize
from aicsimageio import AICSImage
import json
from tqdm import tqdm
import pickle
from parfor import pmap
import pandas as pd
import time
import nd2

In [ ]:
overwrite_flag = False
data_root = "/net/trapnell/vol1/home/nlammers/projects/data/morphseq/"
dir_list = ["20231206"]

In [ ]:
import cv2 

def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

def doLap(image, lap_size=3, blur_size=3):

    # YOU SHOULD TUNE THESE VALUES TO SUIT YOUR NEEDS
#     kernel_size = 5  # Size of the laplacian window
#     blur_size = 5  # How big of a kernal to use for the gaussian blur
    # Generally, keeping these two values the same or very close works well
    # Also, odd numbers, please...

    blurred = cv2.GaussianBlur(image, (blur_size, blur_size), 0)
    return cv2.Laplacian(blurred, cv2.CV_64F, ksize=lap_size)

In [ ]:
read_dir_root = os.path.join(data_root, 'raw_image_data', 'YX1') 

d = 0
# initialize dictionary to metadata
sub_name = dir_list[d]
dir_path = os.path.join(read_dir_root, sub_name, "")

# Read in  metadata object
image_list = sorted(glob.glob(dir_path + "*.nd2")) 
if len(image_list) > 1:
    raise Exception("Multiple nd2 files found in " + sub_name + ". Please move extra nd2 files to a subfolder." )
elif len(image_list) == 0:
    raise Exception("No nd2 files found in " + sub_name)

# Read in experiment metadata 
print(f"Processing {sub_name}...")

imObject= nd2.ND2File(image_list[0])
im_shape = imObject.shape
n_time_points = im_shape[0]
n_wells = im_shape[1]
n_z_slices = im_shape[2]
print(n_z_slices)


In [ ]:
from matplotlib import pyplot as plt

rs_res = np.asarray([5.66, 5.66])

# pull dask array
im_array_dask = imObject.to_dask()

# extract frame times
time_int = 10
well_int = 1

# get image resolution
voxel_size = imObject.voxel_size()
voxel_yx = np.asarray([voxel_size[1], voxel_size[0]])
rs_factor = np.divide(voxel_yx, rs_res)

rs_dims_yx = np.round(np.multiply(np.asarray(im_shape[3:]), rs_factor)).astype(int)
# resample images to a standardized resolution

data_zyx = im_array_dask[time_int, well_int, :, :, :].compute()

data_zyx_rs = resize(data_zyx, (data_zyx.shape[0], rs_dims_yx[0], rs_dims_yx[1]), preserve_range=True)
# data_zyx_rs = data_zyx_rs.astype(data_zyx.dtype)

In [ ]:
plt.imshow(np.squeeze(data_zyx[12,:, :]))
plt.show()

In [ ]:
# data_zyx_rs = resize(data_zyx, (data_zyx.shape[0], rs_dims_yx[0], rs_dims_yx[1]), preserve_range=True)
# data_zyx_rs = data_zyx

# normalize image to maximize dynamic range
px99 = np.percentile(data_zyx, 99)
data_zyx_rs = data_zyx_rs / px99
data_zyx_rs[data_zyx_rs>1] = 1
data_zyx_rs = np.round(data_zyx_rs * 65535).astype(np.uint16)

# calculate FF image
laps = []
laps_d = []
for i in range(data_zyx_rs.shape[0]):
    laps.append(doLap(data_zyx_rs[i, :, :], lap_size=3, blur_size=3))

laps = np.asarray(laps)
abs_laps = np.absolute(laps)

# calculate full-focus and depth images
ff_image = np.zeros(shape=data_zyx_rs[0].shape, dtype=data_zyx_rs.dtype)
maxima = abs_laps.max(axis=0)
bool_mask = abs_laps == maxima
mask = bool_mask.astype(np.uint8)
for i in range(data_zyx_rs.shape[0]):
    ff_image[np.where(mask[i] == 1)] = data_zyx_rs[i][np.where(mask[i] == 1)]


ff_image = 65535 - ff_image


In [ ]:
np.sum(np.sum(bool_mask, axis=1), axis=1).ravel()

In [ ]:
plt.imshow(ff_image)
# plt.clim(0, 15000)
plt.show()

In [ ]:
io.imsave("test.png", ff_image[500:, :1400])

In [ ]:
ff_image.dtype

# Get paths to data, figures, and latent space outputs
overwrite_flag = False
data_root = "/net/trapnell/vol1/home/nlammers/projects/data/morphseq/"
dir_list = ["20231206"]

In [ ]:
read_dir_root = os.path.join(data_root, 'raw_image_data', 'YX1') 
if write_dir is None:
    write_dir = os.path.join(data_root, 'built_image_data', 'YX1') 
    
# handle paths
if dir_list is None:
    # Get a list of directories
    dir_list_raw = sorted(glob.glob(read_dir_root + "*"))
    dir_list = []
    for dd in dir_list_raw:
        if os.path.isdir(dd):
            dir_list.append(path_leaf(dd))

if rs_res is None:
    rs_res = np.asarray([5.66, 5.66])

# filter for desired directories
dir_indices = [d for d in range(len(dir_list)) if "ignore" not in dir_list[d]]

In [ ]:
os.listdir(model_path)

In [ ]:
import pandas as pd
    
last_training = sorted(os.listdir(model_path))[-2]
    
m_fig_path = os.path.join(model_path, last_training, "figures")
    
# load data frame with results
morph_df = pd.read_csv(os.path.join(m_fig_path, "embryo_stats_df.csv"), index_col=0)
metric_df = pd.read_csv(os.path.join(m_fig_path, "metric_df.csv"), index_col=0)
meta_df = pd.read_csv(os.path.join(m_fig_path, "meta_summary_df.csv"), index_col=0)
loss_df = pd.read_csv(os.path.join(model_path, last_training, "loss_tracker.csv"), index_col=0)

In [ ]:
import plotly.express as px 
import plotly.graph_objects as go

wik_indices = np.where(morph_df["master_perturbation"]=="wck-AB")[0]

fig = px.scatter(morph_df.iloc[wik_indices], x="UMAP_00", y="UMAP_01", color="predicted_stage_hpf", opacity=0.5)
fig.show()

In [ ]:
fig = px.scatter(morph_df.iloc[wik_indices], x="UMAP_00_bio", y="UMAP_01_bio", color="predicted_stage_hpf", 
                 opacity=0.5)
fig.show()

In [ ]:
fig = px.scatter(morph_df.iloc[wik_indices], x="UMAP_00_bio", y="UMAP_01_bio", color="train_cat", 
                 opacity=0.5)
fig.show()

In [ ]:
fig = px.scatter(morph_df.iloc[wik_indices], x="UMAP_00_n", y="UMAP_01_n", color="predicted_stage_hpf", 
                 opacity=0.5)
fig.show()

Despite the imposition of a metric constraint on the biological latent variables, they still quite obviously encode orientational differences. Why?

It could be that the model doing a good job with cosine distance, but that this does not translate to the euclidean space that is being read out (and warped in obscure ways) by the UMAP compression.

It is also possible that the metric loss is simply outcompeted by other terms.

In [ ]:
from sklearn.metrics import pairwise_distances

metric_df0 = metric_df.iloc[np.where(metric_df["contrast_id"]==0)]

fig = go.Figure()

fig.add_trace(go.Histogram(x=metric_df0["cos_all"], histnorm="probability", name="all latent variables"))
fig.add_trace(go.Histogram(x=metric_df0["cos_bio"], histnorm="probability", name="biological partition"))
fig.add_trace(go.Histogram(x=metric_df0["cos_nbio"], histnorm="probability", name="non-biological partition"))
fig.update_layout(title="Cosine similarity scores between pairs of transformed images")
fig.show()


Clearly, SOMETHING is being learned such that the distance between transformed versions of the same image look more similar in the biological partition (according to the cosine metric) than in the non-biological one.

In [ ]:
# I think we need to normalize by number of bio and non-bio variables
n_bio = 90
n_nbio = 10

metric_df0.loc[:, "euc_all_norm"] = metric_df0.loc[:, "euc_all"] / np.sqrt(n_bio + n_nbio)
metric_df0.loc[:, "euc_bio_norm"] = metric_df0.loc[:, "euc_bio"] / np.sqrt(n_bio)
metric_df0.loc[:, "euc_nbio_norm"] = metric_df0.loc[:, "euc_nbio"] / np.sqrt(n_nbio)

fig = go.Figure()

fig.add_trace(go.Histogram(x=metric_df0["euc_all_norm"], histnorm="probability", name="all latent variables"))
fig.add_trace(go.Histogram(x=metric_df0["euc_bio_norm"], histnorm="probability", name="biological partition"))
fig.add_trace(go.Histogram(x=metric_df0["euc_nbio_norm"], histnorm="probability", name="non-biological partition"))
fig.update_layout(title="Average Euclidean distance between pairs of transformed images")
fig.show()

In [ ]:
fig = px.line(loss_df, x="epoch", y="ntxent_loss", color="train_cat", markers=False,
             labels={"ntxent_loss":"contrastive loss"},
             log_y=True)
fig.show()

In [ ]:
fig = px.line(loss_df, x="epoch", y="recon_loss", color="train_cat", markers=False, #, log_y=True,
             labels={"recon_loss":"reconstruction loss (MSE)"},
             log_y=True)

# fig.update_xaxes(range=[0, 250])
fig.update_yaxes(range=[2, 3])

fig.show()

In [ ]:
fig = px.line(loss_df, x="epoch", y="reg_loss", color="train_cat", markers=False, #, log_y=True,
             labels={"reg_loss":"Gaussian prior loss (KLD)"})

# fig.update_xaxes(range=[0, 250])
# fig.update_yaxes(range=[0, 650])

fig.show()

In [ ]:
import plotly.express as px 
import plotly.graph_objects as go

gdf3_indices = np.where(morph_df["master_perturbation"]=="gdf3")[0]

fig = px.scatter(morph_df.iloc[gdf3_indices], x="UMAP_00", y="UMAP_01", color="predicted_stage_hpf", 
                 opacity=0.5, template="plotly")

fig.add_trace(go.Scatter(x=morph_df["UMAP_00_bio"].iloc[wik_indices], y=morph_df["UMAP_01_bio"].iloc[wik_indices],
              mode="markers", marker=dict(color="blue", opacity=0.1))
             )

fig.show()